### Regression and One-click deployment example

In this Notebook I walk through all the steps needed to develop a **regression model** and save the model to the OCI Data Science **Model Catalog**.

After the model have been saved, I deploy the model as a **REST** service.

In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

# the dataset used for the example
from sklearn.datasets import fetch_california_housing

from sklearn.model_selection import train_test_split

# the GBM used
import xgboost as xgb

# to use ADSTuner
from ads.hpo.search_cv import ADSTuner
from ads.hpo.stopping_criterion import *
from ads.hpo.distributions import *

# to save to Modeel catalog
import pickle
import os
from ads import set_auth
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
from ads.common.model_metadata import (MetadataCustomCategory,
                                       UseCaseType)

### some utility functions

In [2]:
# functions
def get_general_info(data_df):
    print(f"There are: {len(data_df.columns)} columns in the dataset")
    print()
    print(
        "The list of column names, in alphabetical order:",
        sorted(list(data_df.columns)),
    )
    print()
    print(f"There are {data_df.shape[0]} records in the dataset")
    print()
    
    return

# well you have to decide a threshold in term of a fraction
# to decide if the col is categorical
FRAC = 0.1

def analyze_df(data_df):
    # it is ok to use isna, isnull is an alias of isna
    missing_val = data_df.isna().sum()

    # cardinality

    THR = data_df.shape[0] * FRAC

    list_card = []
    list_cat = []
    list_dtypes = []
    list_num_zeros = []

    for col in data_df.columns:
        # count the # of distinct values
        n_distinct = data_df[col].nunique()
        list_card.append(n_distinct)
        
        # is categorical is decide on this rule
        if n_distinct < THR:
            # categorical
            list_cat.append("Yes")
        else:
            list_cat.append("No")

        list_dtypes.append(data_df[col].dtype)

    # build the results DF
    result_df = pd.DataFrame(
        {
            "col_name": list(data_df.columns),
            "missing_vals": missing_val,
            "cardinality": list_card,
            "is_categorical": list_cat,
            "data_type": list_dtypes,
        },
        index=None,
    )

    # if you don't want cols as index
    result_df.reset_index(drop=True, inplace=True)

    return result_df

def show_tuner_results(tuner):

    # to count completed
    result_df = tuner.trials[tuner.trials["state"] == "COMPLETE"].sort_values(
        by=["value"], ascending=False
    )

    print("ADSTuner session results:")
    print(f"ADSTuner has completed {result_df.shape[0]} trials")
    print()
    print(f"The best trial is the #: {tuner.best_index}")
    print(f"Parameters for the best trial are: {tuner.best_params}")
    print(f"The metric used to optimize is: {tuner.scoring_name}")
    print(f"The best score is: {round(tuner.best_score, 4)}")

### Load the dataset

In [3]:
# load the dataset
housing = fetch_california_housing(as_frame=True)

orig_df = housing.frame

In [4]:
orig_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


### some EDA

In [5]:
get_general_info(orig_df)

analyze_df(orig_df)

There are: 9 columns in the dataset

The list of column names, in alphabetical order: ['AveBedrms', 'AveOccup', 'AveRooms', 'HouseAge', 'Latitude', 'Longitude', 'MedHouseVal', 'MedInc', 'Population']

There are 20640 records in the dataset



,col_name,missing_vals,cardinality,is_categorical,data_type
0,MedInc,0,12928,No,float64
1,HouseAge,0,52,Yes,float64
2,AveRooms,0,19392,No,float64
3,AveBedrms,0,14233,No,float64
4,Population,0,3888,No,float64
5,AveOccup,0,18841,No,float64
6,Latitude,0,862,Yes,float64
7,Longitude,0,844,Yes,float64
8,MedHouseVal,0,3842,No,float64


In [6]:
# In this example I'll use all the columns (ex MedHouseVal) as features, except Lat, Long, to simplify

TARGET = "MedHouseVal"
all_cols = list(orig_df.columns)
cols_to_drop = ['Latitude', 'Longitude']

cat_cols = ['HouseAge']

# take care, I have sorted
FEATURES = sorted(list(set(all_cols) - set([TARGET])- set(cols_to_drop)))

# for LightGBM
cat_columns_idxs = [i for i, col in enumerate(FEATURES) if col in cat_cols]

FEATURES

['AveBedrms', 'AveOccup', 'AveRooms', 'HouseAge', 'MedInc', 'Population']

In [7]:
# the only important thing is that we have 1 categorical column: HouseAge

# we will code categorical as integer starting from zero
# in this case it is easy, since the minimum is 1... so we need only to subtract 1

In [8]:
used_df = orig_df.copy()

used_df['HouseAge'] = used_df['HouseAge'] - 1.

used_df['HouseAge'] = used_df['HouseAge'].astype(int)
used_df['HouseAge'] = used_df['HouseAge'].astype("category")

In [9]:
# let's make a simple train/test split
X = used_df[FEATURES].values
y = used_df[TARGET].values

TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=1)

### HPO using ADSTuner

In [10]:
STUDY_NAME = "xgb001"
FOLDS = 5
TIME_BUDGET = 1800

#
# Here we define the strategy, the space for hyper-parameters we want to explore
#
params = {
    "n_estimators": CategoricalDistribution([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
    "learning_rate": LogUniformDistribution(low=1e-4, high=1e-2),
    "max_depth": IntUniformDistribution(5, 10),
}

alg_reg = xgb.XGBRegressor()

tuner = ADSTuner(
    alg_reg, cv=FOLDS, strategy=params, study_name=STUDY_NAME,  scoring='neg_mean_absolute_error', n_jobs=8)

tuner.tune(X_train, y_train, exit_criterion=[TimeBudget(TIME_BUDGET)])

[I 2022-03-30 12:38:02,130] A new study created in RDB with name: xgb001


  FutureWarning,



In [58]:
# get the status to see if completed
print(f"The tuner status is: {tuner.get_status()}")

print(f"Remaining time is: {round(tuner.time_remaining, 1)} sec.")

The tuner status is: State.RUNNING
Remaining time is: 0 sec.


In [60]:
show_tuner_results(tuner)

ADSTuner session results:
ADSTuner has completed 83 trials

The best trial is the #: 67
Parameters for the best trial are: {'learning_rate': 0.008062192140586195, 'max_depth': 7, 'n_estimators': 600}
The metric used to optimize is: neg_mean_absolute_error
The best score is: -0.4555


In [61]:
# look only at completed trials, sorted with best on top. Metric chosen is in the value col.
result_df = tuner.trials[tuner.trials["state"] == "COMPLETE"].sort_values(
    by=["value"], ascending=False
)

result_df.head(10)

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_n_estimators,user_attrs_mean_fit_time,user_attrs_mean_score_time,...,user_attrs_metric,user_attrs_split0_test_score,user_attrs_split1_test_score,user_attrs_split2_test_score,user_attrs_split3_test_score,user_attrs_split4_test_score,user_attrs_std_fit_time,user_attrs_std_score_time,user_attrs_std_test_score,state
67,67,-0.455539,2022-03-30 13:03:41.617749,2022-03-30 13:06:13.279106,0 days 00:02:31.661357,0.008062,7,600,30.261721,0.023631,...,neg_mean_absolute_error,-0.463305,-0.453317,-0.462680,-0.444842,-0.453550,0.276152,0.002260,0.006850,COMPLETE
65,65,-0.455600,2022-03-30 13:02:02.259812,2022-03-30 13:04:38.269984,0 days 00:02:36.010172,0.008132,7,600,31.133898,0.021660,...,neg_mean_absolute_error,-0.463508,-0.453457,-0.463199,-0.444369,-0.453467,0.247324,0.003108,0.007149,COMPLETE
46,46,-0.455620,2022-03-30 12:55:29.020873,2022-03-30 12:58:49.886188,0 days 00:03:20.865315,0.006957,7,800,40.102214,0.023448,...,neg_mean_absolute_error,-0.463604,-0.453247,-0.463339,-0.444868,-0.453043,0.250521,0.003914,0.007088,COMPLETE
48,48,-0.455697,2022-03-30 12:56:51.065989,2022-03-30 13:00:18.875111,0 days 00:03:27.809122,0.006662,7,800,41.489235,0.027047,...,neg_mean_absolute_error,-0.463189,-0.453949,-0.463459,-0.444863,-0.453027,1.430283,0.003031,0.006985,COMPLETE
35,35,-0.455748,2022-03-30 12:50:24.302057,2022-03-30 12:54:26.640432,0 days 00:04:02.338375,0.006743,7,1000,48.389825,0.026702,...,neg_mean_absolute_error,-0.464020,-0.452147,-0.463840,-0.445094,-0.453638,0.769328,0.001955,0.007278,COMPLETE
43,43,-0.455786,2022-03-30 12:54:39.527416,2022-03-30 12:58:23.703982,0 days 00:03:44.176566,0.006926,7,900,44.762748,0.026536,...,neg_mean_absolute_error,-0.463451,-0.453070,-0.463624,-0.445560,-0.453226,0.306709,0.003335,0.006909,COMPLETE
81,81,-0.455798,2022-03-30 13:07:46.504414,2022-03-30 13:09:41.411129,0 days 00:01:54.906715,0.008815,7,600,22.920653,0.019330,...,neg_mean_absolute_error,-0.462998,-0.453342,-0.463794,-0.445577,-0.453282,5.450089,0.005012,0.006821,COMPLETE
49,49,-0.455817,2022-03-30 12:57:31.726466,2022-03-30 13:01:01.839370,0 days 00:03:30.112904,0.006607,7,800,41.954002,0.019136,...,neg_mean_absolute_error,-0.463670,-0.453175,-0.464256,-0.445350,-0.452634,1.544820,0.001686,0.007205,COMPLETE
70,70,-0.455860,2022-03-30 13:04:38.290145,2022-03-30 13:07:09.492835,0 days 00:02:31.202690,0.008076,7,600,30.170967,0.021679,...,neg_mean_absolute_error,-0.463394,-0.453560,-0.463703,-0.445241,-0.453404,0.130327,0.005552,0.006962,COMPLETE
71,71,-0.455903,2022-03-30 13:04:52.349949,2022-03-30 13:07:23.618740,0 days 00:02:31.268791,0.007936,7,600,30.182391,0.021059,...,neg_mean_absolute_error,-0.463728,-0.453796,-0.463474,-0.444684,-0.453834,0.124425,0.001141,0.007115,COMPLETE


### Train with best params

In [62]:
%%time
### train with best params

model = xgb.XGBRegressor(**tuner.best_params)

model.fit(X_train, y_train)

CPU times: user 1h 20min 33s, sys: 11.2 s, total: 1h 20min 45s
Wall time: 2min 51s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.008062192140586195,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=600, n_jobs=32,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

### Save the model

In [63]:
# save the model
MODEL_FILE_NAME = "model.pkl"

pickle.dump(model, open(MODEL_FILE_NAME, "wb"))

In [64]:
# test if it loads correctly
loaded_model = pickle.load(open(MODEL_FILE_NAME, "rb"))

loaded_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.008062192140586195,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=600, n_jobs=32,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

### Prepare for Model Catalog

In [188]:
PATH_ARTEFACT = f"./model-files"

if not os.path.exists(PATH_ARTEFACT):
    os.mkdir(PATH_ARTEFACT)

In [189]:
# if we pass x_test, y_test generates in metadata also the schema
artifact = prepare_generic_model(model=model, model_path=PATH_ARTEFACT,
                                 force_overwrite=True, 
                                 data_science_env=True,
                                 X_sample=X_test,
                                 y_sample=y_test,
                                 use_case_type=UseCaseType.REGRESSION)

# add the model file to the PATH_ARTEFACT directory
pickle.dump(model, open(PATH_ARTEFACT + "/" + MODEL_FILE_NAME, "wb"))

# to set the serialization format in metadata
artifact.reload(model_file_name=MODEL_FILE_NAME)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

### Customize score.py

In [190]:
%%writefile {PATH_ARTEFACT}/score.py

import pandas as pd
import numpy as np

from xgboost import XGBClassifier

import json
import os
import pickle

import io
import logging 

# logging configuration - OPTIONAL 
logging.basicConfig(format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger_pred = logging.getLogger('model-prediction')
logger_pred.setLevel(logging.INFO)
logger_feat = logging.getLogger('input-features')
logger_feat.setLevel(logging.INFO)

model_name = 'model.pkl'

# to enable/disable detailed logging
DEBUG = True

"""
   Inference script. This script is used for prediction by scoring server when schema is known.
"""

def load_model(model_file_name=model_name):
    """
    Loads model from the serialized format

    Returns
    -------
    model:  a model instance on which predict API can be invoked
    """
    
    model_dir = os.path.dirname(os.path.realpath(__file__))
    contents = os.listdir(model_dir)
    
    # Load the model from the model_dir using the appropriate loader
    
    if model_file_name in contents:
        with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), model_file_name), "rb") as file:
            model = pickle.load(file) 
            logger_pred.info("Loaded the model !!!")
                
    else:
        raise Exception('{0} is not found in model directory {1}'.format(model_file_name, model_dir))
    
    return model

def pre_inference(data):
    """
    Preprocess data

    Parameters
    ----------
    data: Data format as expected by the predict API of the core estimator.

    Returns
    -------
    data: Data format after any processing.

    """
    logger_pred.info("Preprocessing...")
    
    return data

def post_inference(yhat):
    """
    Post-process the model results

    Parameters
    ----------
    yhat: Data format after calling model.predict.

    Returns
    -------
    yhat: Data format after any processing.

    """
    logger_pred.info("Postprocessing output...")
    
    return yhat

def predict(data, model=load_model()):
    """
    Returns prediction given the model and data to predict

    Parameters
    ----------
    model: Model instance returned by load_model API
    data: Data format as expected by the predict API of the core estimator. For eg. in case of sckit models it could be numpy array/List of list/Pandas DataFrame

    Returns
    -------
    predictions: Output from scoring server
        Format: {'prediction': output from model.predict method}

    """
    
    logger_pred.info("In function predict...")
    
    # some check
    assert model is not None, "Model is not loaded"
    
    x = pd.read_json(io.StringIO(data)).values
    
    if DEBUG:
        logger_feat.info("Logging features")
        logger_feat.info(x)
    
    # preprocess data (for example normalize features)
    x = pre_inference(x)

    logger_pred.info("Invoking model......")
    
    # compute predictions (binary, from model)
    preds = model.predict(x)
    
    # to avoid not JSON serialiable error (np.array is not)
    preds = preds.tolist()
    
    # post inference not needed
    return {'prediction': preds}

Overwriting ./model-files/score.py


### Model introspection

In [191]:
artifact.introspect()

['model.pkl', 'output_schema.json', 'input_schema.json', 'runtime.yaml', 'test_json_output.json', '.ipynb_checkpoints', '__pycache__', 'score.py']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Failed,"In runtime.yaml, the key MODEL_DEPLOYMENT.INFERENCE_ENV_PATH does not exist."
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


In [192]:
# check all the info that will be saved to the Model Catalog
artifact.metadata_taxonomy.to_dataframe()

,Key,Value
0,Algorithm,XGBRegressor
1,ArtifactTestResults,"{'score_py': {'key': 'score_py', 'category': 'Mandatory Files Check', 'description': 'Check that the file ""score.py"" exists and is in the top level directory of the artifact directory', 'error_msg': 'The file 'score.py' is missing.', 'success': True}, 'runtime_yaml': {'category': 'Mandatory Files Check', 'description': 'Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory', 'error_msg': 'The file 'runtime.yaml' is missing.', 'success': True}, 'score_syntax': {'category': 'score.py', 'description': 'Check for Python syntax errors', 'error_msg': 'There is Syntax error in score.py: ', 'success': True}, 'score_load_model': {'category': 'score.py', 'description': 'Check that load_model() is defined', 'error_msg': 'Function load_model is not present in score.py.', 'success': True}, 'score_predict': {'category': 'score.py', 'description': 'Check that predict() is defined', 'error_msg': 'Function predict is not present in score.py.', 'success': True}, 'score_predict_data': {'category': 'score.py', 'description': 'Check that the only required argument for predict() is named ""data""', 'error_msg': 'The predict function in score.py must have a formal argument named 'data'.', 'success': True}, 'score_predict_arg': {'category': 'score.py', 'description': 'Check that all other arguments in predict() are optional and have default values', 'error_msg': 'All formal arguments in the predict function must have default values, except that 'data' argument.', 'success': True}, 'runtime_version': {'category': 'runtime.yaml', 'description': 'Check that field MODEL_ARTIFACT_VERSION is set to 3.0', 'error_msg': 'In runtime.yaml, the key MODEL_ARTIFACT_VERSION must be set to 3.0.', 'success': True}, 'runtime_env_python': {'category': 'conda_env', 'description': 'Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher', 'error_msg': 'In runtime.yaml, the key MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION must be set to a value of 3.6 or higher.', 'success': True, 'value': '3.7.12'}, 'runtime_env_path': {'category': 'conda_env', 'description': 'Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set', 'error_msg': 'In runtime.yaml, the key MODEL_DEPLOYMENT.INFERENCE_ENV_PATH must have a value.', 'success': True, 'value': 'oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General Machine Learning for CPUs on Python 3.7/1.0/generalml_p37_cpu_v1'}, 'runtime_path_exist': {'category': 'conda_env', 'description': 'Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.', 'error_msg': 'In runtime.yaml, the key MODEL_DEPLOYMENT.INFERENCE_ENV_PATH does not exist.', 'success': False}}"
2,Framework,xgboost
3,FrameworkVersion,1.5.1
4,Hyperparameters,"{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0, 'gpu_id': -1, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.008062192140586195, 'max_delta_step': 0, 'max_depth': 7, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 600, 'n_jobs': 32, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}"
5,UseCaseType,regression


### Some tests on the code before saving to Model Catalog

In [193]:
# %reload_ext autoreload
%load_ext autoreload

%autoreload 2

# add the path of score.py: 

import sys 
sys.path.insert(0, PATH_ARTEFACT)

from score import load_model, predict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
# Load the model to memory 
_ = load_model()

INFO:model-prediction:Loaded the model !!!


In [199]:
# select some records
START = 40
END = 50

predictions_test = predict(json.dumps(X_test[START:END].tolist()), _)

print()
print("*********************************")
print("Tests results:")
print()
print("Predicted:")
print(np.around(predictions_test['prediction'], 3))

print()
print("Expected:")
print(y_test[START:END])
print()
print(f"Computed MAE: {round(np.abs(predictions_test['prediction'] - y_test[START:END]).mean(), 2)}")

INFO:model-prediction:In function predict...
INFO:input-features:Logging features
INFO:input-features:[[1.05042017e+00 4.66386555e+00 5.08403361e+00 1.90000000e+01
  2.24480000e+00 1.66500000e+03]
 [1.02360877e+00 3.37268128e+00 6.10455312e+00 2.50000000e+01
  4.99620000e+00 2.00000000e+03]
 [1.09302326e+00 2.34302326e+00 5.56395349e+00 5.10000000e+01
  3.23440000e+00 4.03000000e+02]
 [1.20202020e+00 2.03030303e+00 4.06734007e+00 4.30000000e+01
  2.61030000e+00 6.03000000e+02]
 [9.26931106e-01 3.05219207e+00 4.34864301e+00 2.60000000e+01
  2.64390000e+00 1.46200000e+03]
 [1.03645008e+00 4.93977813e+00 4.36291601e+00 3.30000000e+01
  3.17130000e+00 3.11700000e+03]
 [1.75000000e+00 4.00000000e+00 1.05000000e+01 2.30000000e+01
  5.87500000e+00 3.20000000e+01]
 [1.03831982e+00 3.28150332e+00 6.07516581e+00 1.40000000e+01
  5.48290000e+00 4.45300000e+03]
 [1.00502513e+00 2.83542714e+00 4.56030151e+00 3.20000000e+01
  3.24690000e+00 2.25700000e+03]
 [1.10979228e+00 3.04451039e+00 4.75667656e

### Save to Model Catalog

In [196]:
# Saving the model artifact to the model catalog.
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
project_id = os.environ['PROJECT_OCID']

set_auth(auth='resource_principal')

#
# Save to Model Catalog
#
catalog_entry = artifact.save(display_name='california-housing1', 
                              description='A model for regression',
                              # to avoid to commit (be careful)
                              ignore_pending_changes=True)

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

artifact:/tmp/saved_model_6b075aa6-efd8-4cfa-a29b-87903b5661f9.zip
